In [ ]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
import cv2
from skimage import color
from skimage import io
from skimage.filters import gaussian
from scipy.ndimage import binary_fill_holes

In [ ]:
def merge_mask(image, mask):
    b = mask
    g = image[:,:,1]
    r = np.zeros_like(image[:,:,0]).astype(np.uint8)
    merged = np.stack([r, g, b], axis=2)
    return merged

In [ ]:
img = io.imread(infile)
image = img.copy()
lower = [10, 15, 0]
upper = [255, 100, 10]

# create NumPy arrays from the boundaries
lower = np.array(lower, dtype="uint8")
upper = np.array(upper, dtype="uint8")

# find the colors within the specified boundaries and apply
# the mask
mask = cv2.inRange(image, lower, upper)
print(mask.dtype, mask.shape)
mask = (binary_fill_holes(mask)).astype(np.uint8)
print(mask.dtype, mask.shape)
output = cv2.bitwise_and(image, image, mask=mask)
ret,thresh = cv2.threshold(mask, 0, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
areaArray = []
for contour in contours:
    area = cv2.contourArea(contour)
    areaArray.append(area)
# first sort the array by area
if len(contours) > 0:
    sorteddata = sorted(zip(areaArray, contours), key=lambda x: x[0], reverse=True)
    largest_contour = sorteddata[0][1]
    output = cv2.fillPoly(mask, pts =[largest_contour], color=255)
    #plt.imshow(output, cmap="gray")
print(output.shape, output.ndim)
plt.imshow(mask, cmap="gray")

In [ ]:
print(image.dtype, image.shape, image.ndim)
print(output.dtype, output.shape, output.ndim)

In [ ]:
merged = merge_mask(image, output)
print(merged.shape, merged.dtype, merged.ndim)

In [ ]:
plt.imshow(mask)

In [ ]:
animal = 'DK37'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/masks'
INPUT = os.path.join(DIR, 'C1', 'thumbnail_colored')
file = '101.tif'
infile = os.path.join(INPUT, file)

In [ ]:
img = io.imread(infile)
print('img',img.dtype, img.shape)
new_img = color.rgb2gray(img)
new_img *= 255 # or any coefficient
new_img = new_img.astype(np.uint8)
new_img[(new_img > 200)] = 0

In [ ]:
lowerbound = 0
upperbound = 255
#all pixels value above lowerbound will  be set to upperbound 
_, thresh = cv2.threshold(new_img.copy(), lowerbound, upperbound, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))
thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel)
#smoothed = gaussian(thresh, 50)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
smoothed = cv2.morphologyEx(thresh, cv2.MORPH_ERODE, kernel)
inverted_thresh = cv2.bitwise_not(smoothed)
filled_thresh = binary_fill_holes(inverted_thresh).astype(np.uint8)
cleaned = cv2.bitwise_and(img,img, mask=filled_thresh)

In [ ]:
images = [cleaned, smoothed, thresh, filled_thresh, img]
_, axs = plt.subplots(len(images),1, figsize=(12, 12))
axs = axs.flatten()
for image, ax in zip(images, axs):
    ax.imshow(image, cmap="gray")
plt.show()